# ML.Net - EntityFrameworkData

In [12]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML" 
#r "nuget:Microsoft.EntityFrameworkCore" 
#r "nuget:Microsoft.EntityFrameworkCore" 
#r "nuget:Microsoft.EntityFrameworkCore.InMemory" 
#r "nuget:Microsoft.EntityFrameworkCore.Tools" 
#r "nuget:Microsoft.EntityFrameworkCore.SqlServer" 

Installed package Microsoft.EntityFrameworkCore.SqlServer version 3.1.5

Installing package Microsoft.EntityFrameworkCore.Tools...

## Using C# Class

In [18]:
using Microsoft.EntityFrameworkCore;
using Microsoft.ML;
using Microsoft.ML.Data;
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;

## Declare data-classes for input data and predictions

In [14]:
public class SalaryData
{
    public int SalaryDataId { get; set; }
    public float YearsExperience { get; set; }

    public float Salary { get; set; }
}

public class SalaryPrediction
{
    [ColumnName("Score")]
    public float PredictedSalary;
}

In [16]:
public partial class MLNetExampleContext : DbContext
{
    public DbSet<SalaryData> Salaries { get; set; }

    public MLNetExampleContext(DbContextOptions<MLNetExampleContext> options)
        : base(options)
    {
    }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        if (!optionsBuilder.IsConfigured)
        {
            optionsBuilder.UseSqlServer("Server=(localdb)\\mssqllocaldb;Database=EFProviders.InMemory;Trusted_Connection=True;ConnectRetryCount=0");
        }
    }
}

In [19]:
private static IEnumerable<SalaryData> WriteAndReadToDatabase(IEnumerable<SalaryData> fileData)
{
    var options = new DbContextOptionsBuilder<MLNetExampleContext>()
                    .UseInMemoryDatabase(databaseName: "TestData")
                    .Options;

    using (var context = new MLNetExampleContext(options))
    {
        foreach (var item in fileData)
        {
            context.Salaries.Add(item);
        }

        var count = context.SaveChanges();

        return context.Salaries.ToList();
    }
}

private static IEnumerable<SalaryData> ReadFromFile(string filePath)
{
    var data = File.ReadAllLines(filePath)
        .Skip(1)
        .Select(l => l.Split(','))
        .Select(i => new SalaryData
        {
            YearsExperience = float.Parse(i[0]),
            Salary = float.Parse(i[1])
        });

    return data;
}

## Evaluate

In [21]:
private static IEnumerable<SalaryData> data;

var fileData = ReadFromFile("./datasets/salary/SalaryData.csv");

data = WriteAndReadToDatabase(fileData);

var context = new MLContext();

var mlData = context.Data.LoadFromEnumerable(data);

var trainTestData = context.Regression.TrainTestSplit(mlData);

var pipeline = context.Transforms.Concatenate("Features", "YearsExperience")
    .Append(context.Transforms.CopyColumns(("Label", "Salary")))
    .Append(context.Regression.Trainers.FastTree());

var model = pipeline.Fit(trainTestData.TrainSet);

var prediction = model.Transform(trainTestData.TestSet);

var metrics = context.Regression.Evaluate(prediction);

var predictionFunc = model.CreatePredictionEngine<SalaryData, SalaryPrediction>(context);

var salaryPrediction = predictionFunc.Predict(new SalaryData { YearsExperience = 11 });

Console.WriteLine($"Prediction - {salaryPrediction.PredictedSalary}");
Console.ReadLine();

Unhandled exception: (11,40): error CS1061: ‘RegressionCatalog’ não contém uma definição para "TrainTestSplit" e não foi possível encontrar nenhum método de extensão "TrainTestSplit" que aceite um primeiro argumento do tipo ‘RegressionCatalog’ (você está se esquecendo de usar uma diretiva ou uma referência de assembly?)
(14,32): error CS7036: Não há nenhum argumento fornecido que corresponde ao parâmetro formal necessário "inputColumnName" de "TransformExtensionsCatalog.CopyColumns(TransformsCatalog, string, string)"
(15,41): error CS1061: ‘RegressionCatalog.RegressionTrainers’ não contém uma definição para "FastTree" e não foi possível encontrar nenhum método de extensão "FastTree" que aceite um primeiro argumento do tipo ‘RegressionCatalog.RegressionTrainers’ (você está se esquecendo de usar uma diretiva ou uma referência de assembly?)
(23,28): error CS1061: ‘TransformerChain<TNewTrans>’ não contém uma definição para "CreatePredictionEngine" e não foi possível encontrar nenhum mét